# Initialization

## Imports

In [1]:
try:
    import blist
    import datetime
    import matplotlib.pyplot as plt
    import numpy as np
    import nevergrad as ng
    import pandas as pd
    import seaborn as sns
    import tqdm
    import os
    
    from backtesting import Backtest, Strategy
    from backtesting.lib import crossover

except:
    !pip install seaborn
    !pip install nevergrad
    !pip install blist
    !pip install pandas
    !pip install np
    !pip install backtesting
    !pip install tqdm
    !pip install matplotlib
    !echo $PYTHONPATH
    

/home/ehsan/.local/lib/python3.8/site-packages/backtesting/_plotting.py:47: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

## Configs

In [2]:
BITCOIN_PRICE_DATA_PATH = os.path.expanduser('/home/ehsan/Documents/university/notebooks/crypto/data/2019-2021_1m_BTC_ohlcv.csv')
START_DATE_TIME = pd.to_datetime('2018-01-01 00:00:00')
FINISH_DATE_TIME = pd.to_datetime('2020-04-22 00:00:00')
ALPHAS = [1000]

class FeatureNames:
    def __init__(self):
        self.date_time = 'date_time'
        self.time = 'open_time'
        self.open = 'open'
        self.high = 'high'
        self.low = 'low'
        self.close = 'close'
        self.time_unit = 'ms'
        self.volume = 'volume'
        
features = FeatureNames()

## Utils

In [3]:
def get_column(input_data):
    return input_data

# Data preparations

## Read data

In [4]:
bitcoin_price_df = pd.read_csv(BITCOIN_PRICE_DATA_PATH)

In [5]:
bitcoin_price_df.head()

,exchange,symbol,open_time,open,high,low,close,volume
0,Binance,BTC/USDT,1546300800000,3701.23,3703.72,3701.09,3702.46,17.100110
1,Binance,BTC/USDT,1546300860000,3702.44,3702.63,3695.66,3697.04,23.700604
2,Binance,BTC/USDT,1546300920000,3699.42,3702.04,3696.08,3698.14,14.488615
3,Binance,BTC/USDT,1546300980000,3697.49,3698.19,3695.97,3696.51,8.499966
4,Binance,BTC/USDT,1546301040000,3697.20,3697.62,3695.00,3696.32,21.782886


## Cleaning the data

In [6]:
bitcoin_price_df[features.date_time] = pd.to_datetime(bitcoin_price_df[features.time], unit=features.time_unit)
bitcoin_price_df.set_index(features.date_time, inplace=True)
bitcoin_price_df.fillna(method='ffill', inplace=True)
filtered_bitcoin_price_df = bitcoin_price_df[START_DATE_TIME:FINISH_DATE_TIME].drop(columns=[features.time])
filtered_bitcoin_price_df.head()

,exchange,symbol,open,high,low,close,volume
date_time,,,,,,,
2019-01-01 00:00:00,Binance,BTC/USDT,3701.23,3703.72,3701.09,3702.46,17.100110
2019-01-01 00:01:00,Binance,BTC/USDT,3702.44,3702.63,3695.66,3697.04,23.700604
2019-01-01 00:02:00,Binance,BTC/USDT,3699.42,3702.04,3696.08,3698.14,14.488615
2019-01-01 00:03:00,Binance,BTC/USDT,3697.49,3698.19,3695.97,3696.51,8.499966
2019-01-01 00:04:00,Binance,BTC/USDT,3697.20,3697.62,3695.00,3696.32,21.782886


## Adding handcrafted features

### Alpha

In [7]:
def get_alpha(value_list, window_len):
    value = value_list[0]
    values = blist.blist([value])
    sorted_values = blist.sortedlist(values)
    value_len = 1
    output_values = [0]
    for value in value_list[1:]:
        values.append(value)
        sorted_values.add(value)
        value_len += 1
        if value_len > window_len:
            first_value_in_the_list = values[0]
            values.remove(first_value_in_the_list)
            sorted_values.remove(first_value_in_the_list)
            value_len -= 1
        output_values.append((sorted_values.index(value) + 1) / value_len)
    return output_values

bitcoin_price_with_alpha_df = filtered_bitcoin_price_df.copy()
for alpha in ALPHAS:
    bitcoin_price_with_alpha_df[f'alpha{alpha}'] = (
        get_alpha(filtered_bitcoin_price_df[features.close], window_len=alpha)
    )


In [8]:
%matplotlib
(
    bitcoin_price_with_alpha_df[[features.close] + ['alpha'+str(x) for x in ALPHAS]]
).plot(subplots=True, sharex=True)


Using matplotlib backend: GTK3Agg


array([<AxesSubplot:xlabel='date_time'>, <AxesSubplot:xlabel='date_time'>],
      dtype=object)

/home/ehsan/.local/lib/python3.8/site-packages/ipykernel/gui/gtk3embed.py:59: RuntimeWarning: coroutine 'Kernel.do_one_iteration' was never awaited
  self.kernel.do_one_iteration()


# Back testing functions

## Re-sampling

In [ ]:
aggregation_dict = {
    features.open: 'first',  
    features.high: 'max', 
    features.low: 'min', 
    features.close: 'last',
    self.volume: 'sum'
}
print(aggregation_dict)
for alpha in ALPHAS:
    aggregation_dict[f'alpha{alpha}'] = 'last'

ohlcv_df = (
    bitcoin_price_with_alpha_df
    .resample('1H')
    .agg(
        aggregation_dict
    )
)
ohlcv_df

In [ ]:
aggregation_dict

## Define and train models

In [ ]:
ohlcv_df['change'] = -ohlcv_df[features.close].shift(1) + ohlcv_df[features.close]
ohlcv_df.head()

## Define Strategy 

In [ ]:

class SingleAlphaStrategy(Strategy):
    buy_level = 0.98
    sell_level = 0.02
    window_size = 100
    
    def init(self):
        self.alpha = self.I(get_alpha, self.data.Close, self.window_size)
        self.price = self.I(get_column, self.data.Close)
        self.max_pl = 0
        
    def next(self):
        price = self.price[-1]
        self.max_pl = max(self.max_pl, self.position.pl)

        if (self.alpha[-1] > self.buy_level) & (self.position.size == 0):
            self.buy()
            self.max_pl = 0
        
        if self.position.size != 0:
            if (self.position.pl < self.max_pl * 0.5) | (self.alpha[-1] < self.sell_level):
                self.position.close()
                self.max_pl = 0


class ScalperStrategy(Strategy):
    buy_level = 0.98
    sell_level = 0.02
    window_size = 100
    
    def init(self):
        self.alpha = self.I(get_alpha, self.data.Close, self.window_size)
        self.price = self.I(get_column, self.data.Close)
        self.max_pl = 0
        
    def next(self):
        price = self.price[-1]
        self.max_pl = max(self.max_pl, self.position.pl)

        if (self.alpha[-1] > self.buy_level) & (self.position.size == 0):
            self.buy(limit=price*1.01, sl=price*0.97)
            self.max_pl = 0
        
        if self.position.size != 0:
            if (self.position.pl < self.max_pl * 0.5) | (self.alpha[-1] < self.sell_level):
                self.position.close()
                self.max_pl = 0
          
        
bt = Backtest(ohlcv_df, SingleAlphaStrategy)
stats = bt.run()
bt.plot()

In [ ]:

def get_output(
    inp_buy_alpha, inp_sell_alpha, inp_window_size, inp_sl, inp_tp, plot=False, df=ohlcv_df
):

    positions = [0]
    class SingleAlphaStrategy(Strategy):
        buy_alpha=inp_buy_alpha
        sell_alpha=inp_sell_alpha
        window_size=inp_window_size
        sl=inp_sl
        tp=inp_tp
        
        def init(self):
            self.alpha = self.I(get_alpha, self.data.Close, self.window_size)
            self.price = self.I(get_column, self.data.Close)
            self.max_pl = 0

        def next(self):
            price = self.price[-1]
            self.max_pl = max(self.max_pl, self.position.pl)

            if (self.alpha[-1] > self.buy_alpha) & (self.position.size == 0):
                self.buy(sl=self.sl*price, tp=self.tp*price)
                self.max_pl = 0

            if self.position.size != 0:
                if (self.max_pl - self.position.pl < (self.sl - 1))|(self.alpha[-1] < self.sell_alpha):
                    self.position.close()
                    self.max_pl = 0
            positions.append(self.position.is_long * 1)
            

    bt = Backtest(df, SingleAlphaStrategy, commission=0.001)
    stats = bt.run()
    if plot:
        bt.plot()
        df['status'] = positions
        
        return stats, df
    return -1 * stats.__getitem__('Return [%]')

stats, df = (
    get_output(
        inp_buy_alpha=0.9,
        inp_sell_alpha=0.5,
        inp_window_size=20,
        inp_sl=0.98,
        inp_tp=1.02,
        plot=True
    )
)

In [ ]:
print(set(df['status']))
print(stats)

### Optimization

In [ ]:
%%time

parametrization = ng.p.Instrumentation(
    inp_buy_alpha=ng.p.Scalar(lower=0, upper=1),
    inp_sell_alpha=ng.p.Scalar(lower=0, upper=1),

    # an integer from 1 to 1000
    inp_window_size=48, #ng.p.Scalar(lower=1, upper=240).set_integer_casting(),
    # either "conv" or "fc"
    inp_sl=ng.p.Scalar(lower=0.8, upper=1),
    inp_tp=ng.p.Scalar(lower=1.02, upper=1.1),
)

optimizer = ng.optimizers.NGOpt(parametrization=parametrization, budget=600)
recommendation = optimizer.minimize(get_output)

# show the recommended keyword arguments of the function
print(recommendation.kwargs)

In [ ]:
stats, output_df = get_output(**recommendation.kwargs, plot=True)

In [ ]:
set(output_df['status'])
print(stats)

In [ ]:
stats.__getitem__('Max. Drawdown Duration').days

# Get GIPS table

## Total back testing

In [ ]:
class Backtesting:
    def __init__(
        self,
        data_frame,
        start_date_time,
        trading_start_date,
        finish_date_time,
        working_interval
    ):
        self.data = data_frame
        self.start = start_date_time
        self.finish = finish_date_time
        self.interval = working_interval
        self.stats = []
        self.current_time = trading_start_date
        
    def get_train_df(self):
        return (
            self.data[self.start:self.current_time]
        )
    
    def get_interval_df(self):
        return (
            self.data[self.current_time:self.current_time + self.interval]
        )
    
    def train(self, df):
        pass
    
    def get_interval_stats(self, df, parameters):
        pass
    
    def get_all_stats(self):
        while self.current_time < self.finish:
            print((self.finish - self.current_time) / self.interval)
            train_df = self.get_train_df()
            interval_df = self.get_interval_df()
            model = self.train(train_df)
            stats = self.get_interval_stats(interval_df, model)
            self.stats.append(stats)
            self.current_time += self.interval
    

In [ ]:
class SingleAlphaBackTesting(Backtesting):
    def __init__(self, kwargs):
        super().__init__(**kwargs)

    def train(self, df):
        
        ## Defining the optimization function 
        def get_output(inp_buy_alpha, inp_sell_alpha, inp_window_size, inp_sl, inp_tp, df=df):
            
            ## Defining the strategy based on inputs of the function
            class SingleAlphaStrategy(Strategy):
                buy_alpha=inp_buy_alpha
                sell_alpha=inp_sell_alpha
                window_size=inp_window_size
                sl=inp_sl
                tp=inp_tp

                def init(self):
                    self.alpha = self.I(get_alpha, self.data.Close, self.window_size)
                    self.price = self.I(get_column, self.data.Close)

                def next(self):
                    price = self.price[-1]

                    if (self.alpha[-1] > self.buy_alpha) & (~self.position.is_long):
                        self.buy(sl=self.sl*price, tp=self.tp*price)
                        
                    if self.position.is_long:
                        if self.alpha[-1] < self.sell_alpha:
                            self.position.close()
        

            bt = Backtest(df, SingleAlphaStrategy, commission=0.001, cash=10000000)
            stats = bt.run()
            return -1 * stats.__getitem__('Return [%]')
        
        ## Defining the parameters to be optimized
        parametrization = ng.p.Instrumentation(
            inp_buy_alpha=0.98,#ng.p.Scalar(lower=0, upper=1),
            inp_sell_alpha=0.02,#ng.p.Scalar(lower=0, upper=1),
            inp_window_size=ng.p.Scalar(lower=1, upper=100).set_integer_casting(),
            inp_sl=0.8,#ng.p.Scalar(lower=0.8, upper=1),
            inp_tp=1.5,#ng.p.Scalar(lower=1.02, upper=1.5),
        )

        optimizer = ng.optimizers.NGOpt(parametrization=parametrization, budget=10)
        recommendation = optimizer.minimize(get_output)
        return recommendation
    
    def get_interval_stats(self, df, parameters):
        ## Defining the optimization function 
        def get_output(
            inp_buy_alpha, inp_sell_alpha, inp_window_size, inp_sl, inp_tp, df=df):
            
            ## Defining the strategy based on inputs of the function
            class SingleAlphaStrategy(Strategy):
                buy_alpha=inp_buy_alpha
                sell_alpha=inp_sell_alpha
                window_size=inp_window_size
                sl=inp_sl
                tp=inp_tp

                def init(self):
                    self.alpha = self.I(get_alpha, self.data.Close, self.window_size)
                    self.price = self.I(get_column, self.data.Close)

                def next(self):
                    price = self.price[-1]

                    if (self.alpha[-1] > self.buy_alpha) & (~self.position.is_long):
                        self.buy(sl=self.sl*price, tp=self.tp*price)
                        
                    if self.position.is_long:
                        if self.alpha[-1] < self.sell_alpha:
                            self.position.close()
        

            bt = Backtest(df, SingleAlphaStrategy, commission=0.001, cash=10000000)
            stats = bt.run()
            return stats
        stats = get_output(**parameters.kwargs) 
        return stats
        
        
        

In [ ]:
print(START_DATE_TIME, FINISH_DATE_TIME)
args = {
    "start_date_time":START_DATE_TIME,
    "finish_date_time":FINISH_DATE_TIME,
    "trading_start_date":pd.to_datetime('2018-04-22 00:00:00'),
    "data_frame":ohlcv_df,
    "working_interval": datetime.timedelta(weeks=4)
}

In [ ]:
single_alpha_backtester = SingleAlphaBackTesting(args)

single_alpha_backtester.get_all_stats()

In [ ]:
returns = [x.__getitem__('Return [%]') for x in single_alpha_backtester.stats]
bh_returns = [x.__getitem__('Buy & Hold Return [%]') for x in single_alpha_backtester.stats]
ret = 1
print([x for x in zip(returns, bh_returns)])
print(bh_returns)
for x in returns:
    ret *= (1 + x/100)
    
bh_ret = 1
for x in bh_returns:
    bh_ret *= (1 + x/100)

print(ret, bh_ret)
print(single_alpha_backtester.stats)

## Add assets

In [ ]:
import math
def get_annulized_return(interval_returns, interval):
    print([np.float_power(x + 1, 1/interval) for x in interval_returns])
    mean_daily_return = np.mean([np.float_power(x + 1, 1/interval) for x in interval_returns])
    return (np.power((1 + mean_daily_return), 365) - 1)*100

def get_annulized_volatility(interval_returns, interval):
    list_of_returns = [np.float_power(x, 1/interval) for x in interval_returns]
    daily_returns = [x for _ in range(interval) for x in list_of_returns]
    return (np.sqrt(250) * np.std(daily_returns)) * 100

internal_returns = [x.__getitem__('Return [%]') for x in single_alpha_backtester.stats]
interval = 30
print(internal_returns)
print(get_annulized_return(internal_returns, interval))
print(get_annulized_volatility(internal_returns, interval))